<a href="https://colab.research.google.com/github/dano1234/SharedMindsF23/blob/master/Week4/BetweenTwoImages/SD_Between_Images_Deep_Dive.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Variation on [Stable Diffusion Deep Dive Notebook](https://github.com/fastai/diffusion-nbs/blob/master/Stable%20Diffusion%20Deep%20Dive.ipynb)   and [video](https://www.youtube.com/watch?v=0_BBRNYInx8) by
johnowhitaker From the the [Stable Diffusion Course](https://course.fast.ai/Lessons/part2.html) at the amazing [FastAI](https://www.fast.ai/)  you can import into Colabs from this Github
https://github.com/fastai/diffusion-nbs/blob/master/Stable%20Diffusion%20Deep%20Dive.ipynb

# Stable Diffusion Deep Dive

Stable Diffusion is a powerful text-to-image model. There are various websites and tools to make using it as easy as possible. It is also [integrated into the Huggingface diffusers library](https://huggingface.co/blog/stable_diffusion) where generating images can be as simple as:
```python
from diffusers import StableDiffusionPipeline
pipe = StableDiffusionPipeline.from_pretrained("CompVis/stable-diffusion-v1-4", revision="fp16", torch_dtype=torch.float16, use_auth_token=True).to("cuda")
image = pipe("An astronaught scuba diving").images[0]

```

In this notebook we're going to dig into the code behind these easy-to-use interfaces, to see what is going on under the hood. We'll begin by re-creating the functionality above as a scary chunk of code, and then one by one we'll inspect the different components and figure out what they do. By the end of this notebook that same sampling loop should feel like something you can tweak and modify as you like.

## Setup & Imports

You'll need to log into huggingface and accept the terms of the licence for this model - see the [model card](https://huggingface.co/CompVis/stable-diffusion-v1-4) for details. And when you first run this notebook you need to uncomment the following two cells to install the requirements and log in to huggingface with an access token.

In [ ]:
!pip install -q --upgrade transformers==4.25.1 diffusers ftfy accelerate
!pip install fastapi nest-asyncio pyngrok uvicorn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 40.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 76.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.1/258.1 kB 27.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 28.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 77.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 62.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.3/66.3 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 718.7/718.7 kB 28.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.5/59.5 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 7.2 MB/s eta 0:00:00


In [ ]:
from base64 import b64encode

import numpy
import torch
from diffusers import AutoencoderKL, LMSDiscreteScheduler, UNet2DConditionModel
from huggingface_hub import notebook_login

# For video display:
from IPython.display import HTML
from matplotlib import pyplot as plt
from pathlib import Path
from PIL import Image
from torch import autocast
from torchvision import transforms as tfms
from tqdm.auto import tqdm
from transformers import CLIPTextModel, CLIPTokenizer, logging
import os

torch.manual_seed(1)
if not (Path.home()/'.cache/huggingface'/'token').exists(): notebook_login()

# Supress some unnecessary warnings when loading the CLIPTextModel
logging.set_verbosity_error()

# Set device
torch_device = "cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu"
if "mps" == torch_device: os.environ['PYTORCH_ENABLE_MPS_FALLBACK'] = "1"

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


Moving 0 files to the new cache system


0it [00:00, ?it/s]

## Loading the models

This code (and that in the next section) comes from the [Huggingface example notebook](https://colab.research.google.com/github/huggingface/notebooks/blob/main/diffusers/stable_diffusion.ipynb).

This will download and set up the relevant models and components we'll be using. Let's just run this for now and move on to the next section to check that it all works before diving deeper.

If you've loaded a pipeline, you can also access these components using `pipe.unet`, `pipe.vae` and so on.

In this notebook we aren't doing any memory-saving tricks - if you find yourself running out of GPU RAM, look at the pipeline code for inspiration with things like attention slicing, switching to half precision (fp16), keeping the VAE on the CPU and other modifications.

In [ ]:
# Load the autoencoder model which will be used to decode the latents into image space.
vae = AutoencoderKL.from_pretrained("CompVis/stable-diffusion-v1-4", subfolder="vae")

# Load the tokenizer and text encoder to tokenize and encode the text.
tokenizer = CLIPTokenizer.from_pretrained("openai/clip-vit-large-patch14")
text_encoder = CLIPTextModel.from_pretrained("openai/clip-vit-large-patch14")

# The UNet model for generating the latents.
unet = UNet2DConditionModel.from_pretrained("CompVis/stable-diffusion-v1-4", subfolder="unet")

# The noise scheduler
scheduler = LMSDiscreteScheduler(beta_start=0.00085, beta_end=0.012, beta_schedule="scaled_linear", num_train_timesteps=1000)

# To the GPU we go!
vae = vae.to(torch_device)
text_encoder = text_encoder.to(torch_device)
unet = unet.to(torch_device);

## The Autoencoder (AE)

The AE can 'encode' an image into some sort of latent representation, and decode this back into an image. I've wrapped the code for this into a couple of functions here so we can see what this looks like in action:

In [ ]:
def pil_to_latent(input_im):
    # Single image -> single latent in a batch (so size 1, 4, 64, 64)
    with torch.no_grad():
        latent = vae.encode(tfms.ToTensor()(input_im).unsqueeze(0).to(torch_device)*2-1) # Note scaling
    return 0.18215 * latent.latent_dist.sample()

def latents_to_pil(latents):
    # bath of latents -> list of images
    latents = (1 / 0.18215) * latents
    with torch.no_grad():
        image = vae.decode(latents).sample
    image = (image / 2 + 0.5).clamp(0, 1)
    image = image.detach().cpu().permute(0, 2, 3, 1).numpy()
    images = (image * 255).round().astype("uint8")
    pil_images = [Image.fromarray(image) for image in images]
    return pil_images
    # Prep Scheduler
def set_timesteps(scheduler, num_inference_steps):
    scheduler.set_timesteps(num_inference_steps)
    scheduler.timesteps = scheduler.timesteps.to(torch.float32) # minor fix to ensure MPS compatibility, fixed in diffusers PR 3925


In [ ]:
from PIL import Image
from io import BytesIO
import base64
import skimage
import numpy as np
import json

def encodeb64(image) -> str:

    # convert image to bytes
    with BytesIO() as output_bytes:
        PIL_image = Image.fromarray(skimage.img_as_ubyte(image))
        PIL_image.save(output_bytes, 'JPEG') # Note JPG is not a vaild type here
        bytes_data = output_bytes.getvalue()

    # encode bytes to base64 string
    base64_str = str(base64.b64encode(bytes_data), 'utf-8')
    return base64_str


def interpolate(t, v0, v1, DOT_THRESHOLD=0.9995):
  """Helper function to (spherically) interpolate two arrays v1 v2.
  Taken from: https://gist.github.com/karpathy/00103b0037c5aaea32fe1da1af553355
  """
  inputs_are_torch = False
  if not isinstance(v0, np.ndarray):
    inputs_are_torch = True
    input_device = v0.device
    v0 = v0.cpu().numpy()
    v1 = v1.cpu().numpy()
    #print("came in on device")
  #else:
    #print("came in off device")

  dot = np.sum(v0 * v1 / (np.linalg.norm(v0) * np.linalg.norm(v1)))
  if np.abs(dot) > DOT_THRESHOLD:
    v2 = (1 - t) * v0 + t * v1
  else:
    theta_0 = np.arccos(dot)
    sin_theta_0 = np.sin(theta_0)
    theta_t = theta_0 * t
    sin_theta_t = np.sin(theta_t)
    s0 = np.sin(theta_0 - theta_t) / sin_theta_0
    s1 = sin_theta_t / sin_theta_0
    v2 = s0 * v0 + s1 * v1

  if inputs_are_torch:
    v2 = torch.from_numpy(v2).to(input_device)
    #print("were returned to device")

  return v2

In [ ]:
def findBetweenPictures(_prompt,image1,image2,width,height):
  # Encode to the latent space
  encoded1 = pil_to_latent(image1)
  encoded2 = pil_to_latent(image2)
  encoded = interpolate(.7, encoded1, encoded2)
  decoded = latents_to_pil(encoded)[0]
  decoded
  set_timesteps(scheduler, 15)
  noise = torch.randn_like(encoded) # Random noise
  sampling_step = 8 # Equivalent to step 10 out of 15 in the schedule above
  # encoded_and_noised = scheduler.add_noise(encoded, noise, timestep) # Diffusers 0.3 and below
  encoded_and_noised = scheduler.add_noise(encoded, noise, timesteps=torch.tensor([scheduler.timesteps[sampling_step]]))
  latents_to_pil(encoded_and_noised.float())[0] # Display

  # Settings (same as before except for the new prompt)
  prompt = _prompt
  #height = _width                       # default height of Stable Diffusion
  #width = _height                         # default width of Stable Diffusion
  num_inference_steps = 50            # Number of denoising steps
  guidance_scale = 6                  # Scale for classifier-free guidance
  generator = torch.manual_seed(32)   # Seed generator to create the inital latent noise
  batch_size = 1

  # Prep text (same as before)
  text_input = tokenizer(prompt, padding="max_length", max_length=tokenizer.model_max_length, truncation=True, return_tensors="pt")
  with torch.no_grad():
      text_embeddings = text_encoder(text_input.input_ids.to(torch_device))[0]
  max_length = text_input.input_ids.shape[-1]
  uncond_input = tokenizer(
      [""] * batch_size, padding="max_length", max_length=max_length, return_tensors="pt"
  )
  with torch.no_grad():
      uncond_embeddings = text_encoder(uncond_input.input_ids.to(torch_device))[0]
  text_embeddings = torch.cat([uncond_embeddings, text_embeddings])

  # Prep Scheduler (setting the number of inference steps)
  set_timesteps(scheduler, num_inference_steps)

  # Prep latents (noising appropriately for start_step)
  start_step = 10
  start_sigma = scheduler.sigmas[start_step]
  noise = torch.randn_like(encoded)
  latents = scheduler.add_noise(encoded, noise, timesteps=torch.tensor([scheduler.timesteps[start_step]]))
  latents = latents.to(torch_device).float()

  # Loop
  for i, t in tqdm(enumerate(scheduler.timesteps), total=len(scheduler.timesteps)):
      if i >= start_step: # << This is the only modification to the loop we do

          # expand the latents if we are doing classifier-free guidance to avoid doing two forward passes.
          latent_model_input = torch.cat([latents] * 2)
          sigma = scheduler.sigmas[i]
          latent_model_input = scheduler.scale_model_input(latent_model_input, t)

          # predict the noise residual
          with torch.no_grad():
              noise_pred = unet(latent_model_input, t, encoder_hidden_states=text_embeddings)["sample"]

          # perform guidance
          noise_pred_uncond, noise_pred_text = noise_pred.chunk(2)
          noise_pred = noise_pred_uncond + guidance_scale * (noise_pred_text - noise_pred_uncond)

          # compute the previous noisy sample x_t -> x_t-1
          latents = scheduler.step(noise_pred, t, latents).prev_sample

  return latents_to_pil(latents)[0]

In [ ]:
from fastapi import FastAPI
import nest_asyncio
import json
from pyngrok import ngrok
import uvicorn
from fastapi.encoders import jsonable_encoder
from fastapi.responses import JSONResponse
from fastapi.middleware.cors import CORSMiddleware
from pydantic import BaseModel
from fastapi import Request, FastAPI



#this is another way to package the incoming stuff so the documentation will see the structure.  We are not using
class DataIn(BaseModel):
    data: str

app = FastAPI()

origins = [
    "*",
    "http://localhost",
    "http://localhost:8080",
    "http://127.0.0.1:8000",
]

app.add_middleware(
    CORSMiddleware,
    allow_origins=origins,
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"]
)


@app.get("/")
async def who():
  return {"Who": "Are You"}

@app.get("/hello/{name}")
async def hello(name):
  return {"Hello": name}

def read_root():
    return {"Hello": "World!"}

#async def create_item(dataIn: DataIn):  //using basemodel, not us

@app.post("/generateIt/")
async def get_body(request: Request):
  data = await request.json()
  promptFromClient = data["input"]["prompt"];
  width = data["input"]["width"]
  height = data["input"]["height"];
  picture1 = data["input"]["picture1"];
  print("ICTURE!:", picture1)
  picture2 = data["input"]["picture2"];
  PILImage1 = Image.open(BytesIO(base64.b64decode(picture1)))
  PILImage2 = Image.open(BytesIO(base64.b64decode(picture2)))
  resultImage = findBetweenPictures(promptFromClient,PILImage1,PILImage2,width=width,height=height )
  #images = pipe(promptFromClient,width=width,height=height).images
  #b64Image = encodeb64(images[0])
  b64Image = encodeb64(resultImage)
  d = {
    "prompt": promptFromClient,
    "b64Image": b64Image,
  }
  #return jsonify(d)
  json_compatible_item_data = jsonable_encoder(d)
  return JSONResponse(content=json_compatible_item_data)


ngrok_tunnel = ngrok.connect(8000) # if you don't sign up for a domain you can do this.
#OR
#PUT YOUR OWN STUFF IN HERE Gz
#ngrok.set_auth_token("2IHXCSXJHnT0NVg0iQbojzGqfcR_3chN1kK4rE2eFi3Ezd4")
#ngrok_tunnel = ngrok.connect(8000, "http", domain="dano.ngrok.dev")

print('Public URL:', ngrok_tunnel.public_url)
nest_asyncio.apply()
uvicorn.run(app, port=8000)



INFO:     Started server process [391]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


Public URL: https://7d98-34-125-99-80.ngrok.io
INFO:     70.19.88.148:0 - "OPTIONS /generateIt/ HTTP/1.1" 200 OK
ICTURE!: ...


  0%|          | 0/50 [00:00<?, ?it/s]

INFO:     70.19.88.148:0 - "POST /generateIt/ HTTP/1.1" 200 OK
